In [1]:
import pandas as pd
from mlsecu.anomaly_detection_use_case import *
from mlsecu.data_exploration_utils import *
from mlsecu.data_preparation_utils import *
import re
import matplotlib.pyplot as plt

In [2]:
def load_df(name):
    df = pd.read_excel(name)
    sensor_regex = re.compile(r'^(A|F|L|P)IT\d{3}\.Pv$')
    sensor_cols = [col for col in df.columns if sensor_regex.match(col)]
    sensor_cols.insert(0, 't_stamp')
    df = df[sensor_cols]
    return df

def load_dfs():
    names = ['data/22June2020 (1).xlsx', 'data/22June2020 (2).xlsx', 
             'data/29June2020 (1).xlsx', 'data/29June2020 (2).xlsx']
    dfs = [load_df(name) for name in names]
    df = pd.concat(dfs)
    return df

In [3]:
df = load_df('data/22June2020 (2).xlsx')
df.head()

,t_stamp,LIT101.Pv,FIT101.Pv,FIT201.Pv,AIT201.Pv,AIT202.Pv,AIT203.Pv,AIT301.Pv,AIT302.Pv,AIT303.Pv,...,FIT503.Pv,FIT504.Pv,AIT501.Pv,AIT502.Pv,AIT503.Pv,AIT504.Pv,PIT501.Pv,PIT502.Pv,PIT503.Pv,FIT601.Pv
0,2020-06-22 09:00:00,602.412048,0,0.000384,54.633427,7.587029,175.237122,6.739260,209.590881,47.570263,...,0.002305,0,7.31979,257.446800,47.680080,2.345552,28.165855,16.739820,23.663805,0.000256
1,2020-06-22 09:00:01,602.372800,0,0.000384,54.633427,7.587029,175.237122,6.739260,209.590881,47.570263,...,0.002305,0,7.31979,257.446800,47.680080,2.345552,28.165855,16.739820,23.663805,0.000256
2,2020-06-22 09:00:02,602.215800,0,0.000384,54.633427,7.581902,175.237122,6.731353,209.590881,47.570263,...,0.002305,0,7.31979,257.446800,47.680080,2.345552,28.165855,16.755838,23.663805,0.000256
3,2020-06-22 09:00:03,601.901800,0,0.000384,54.633427,7.575173,175.237122,6.731353,209.616486,47.570263,...,0.002305,0,7.31979,257.472443,47.808254,2.653166,28.165855,16.755838,23.663805,0.000256
4,2020-06-22 09:00:04,601.901800,0,0.000384,54.633427,7.569726,175.262756,6.731353,209.616486,47.570263,...,0.002305,0,7.31979,257.574982,47.808254,2.653166,28.165855,16.755838,23.663805,0.000256


In [4]:
df.describe(include='all')

,t_stamp,LIT101.Pv,FIT101.Pv,FIT201.Pv,AIT201.Pv,AIT202.Pv,AIT203.Pv,AIT301.Pv,AIT302.Pv,AIT303.Pv,...,FIT503.Pv,FIT504.Pv,AIT501.Pv,AIT502.Pv,AIT503.Pv,AIT504.Pv,PIT501.Pv,PIT502.Pv,PIT503.Pv,FIT601.Pv
count,3600,3600.000000,3600.0,3600.000000,3600.000000,3600.000000,3600.000000,3600.000000,3600.000000,3600.000000,...,3600.000000,3600.0,3600.000000,3600.000000,3600.000000,3600.000000,3600.000000,3600.000000,3600.000000,3600.000000
mean,2020-06-22 09:29:59.500000256,662.644693,0.0,0.930189,31.776921,7.628528,147.472230,7.468675,129.706063,43.709538,...,0.107635,0.0,7.484559,175.587599,47.477490,3.473586,216.896905,3.261795,193.131444,0.000257
min,2020-06-22 09:00:00,524.887900,0.0,0.000000,17.591642,7.355999,130.633163,6.571548,109.251549,39.631218,...,0.002305,0.0,7.310818,149.756470,46.814920,1.538067,18.056267,1.842181,13.089592,0.000256
25%,2020-06-22 09:14:59.750000128,602.991041,0.0,0.000256,18.072288,7.549539,136.682900,7.495214,110.788139,40.143414,...,0.115866,0.0,7.382835,151.916175,46.911050,1.922584,232.953100,2.050428,207.558960,0.000256
50%,2020-06-22 09:29:59.500000,693.910300,0.0,0.000384,26.371443,7.579179,139.246353,7.573244,112.068634,43.248604,...,0.117018,0.0,7.500834,158.831070,47.551910,1.961036,233.850300,2.050428,208.408100,0.000256
75%,2020-06-22 09:44:59.249999872,694.970100,0.0,2.347996,37.586517,7.692531,166.489362,7.617357,135.091873,47.570263,...,0.117658,0.0,7.576776,185.657517,48.032555,2.037939,234.523200,2.066447,208.984879,0.000256
max,2020-06-22 09:59:59,810.255432,0.0,2.390671,54.921814,7.918995,175.672900,7.758852,209.949417,58.742554,...,0.120603,0.0,7.636055,257.985138,48.288900,101.473984,237.214813,16.755838,211.452200,0.000449
std,NaN,60.609136,0.0,1.147530,14.771534,0.133763,16.604852,0.284367,31.017039,3.582221,...,0.031092,0.0,0.100684,33.600563,0.509980,9.337131,55.897594,3.911960,50.254552,0.000007


In [5]:
print('Rate of undefined values for each column:')
for col in df.columns:
    print(f'{col}: {df[col].isna().sum() / len(df[col])}')

Rate of undefined values for each column:
t_stamp: 0.0
LIT101.Pv: 0.0
FIT101.Pv: 0.0
FIT201.Pv: 0.0
AIT201.Pv: 0.0
AIT202.Pv: 0.0
AIT203.Pv: 0.0
AIT301.Pv: 0.0
AIT302.Pv: 0.0
AIT303.Pv: 0.0
LIT301.Pv: 0.0
FIT301.Pv: 0.0
LIT401.Pv: 0.0
FIT401.Pv: 0.0
AIT401.Pv: 0.0
AIT402.Pv: 0.0
FIT501.Pv: 0.0
FIT502.Pv: 0.0
FIT503.Pv: 0.0
FIT504.Pv: 0.0
AIT501.Pv: 0.0
AIT502.Pv: 0.0
AIT503.Pv: 0.0
AIT504.Pv: 0.0
PIT501.Pv: 0.0
PIT502.Pv: 0.0
PIT503.Pv: 0.0
FIT601.Pv: 0.0


In [6]:
all_df = load_dfs()
all_df.head()

,t_stamp,LIT101.Pv,FIT101.Pv,FIT201.Pv,AIT201.Pv,AIT202.Pv,AIT203.Pv,AIT301.Pv,AIT302.Pv,AIT303.Pv,...,FIT503.Pv,FIT504.Pv,AIT501.Pv,AIT502.Pv,AIT503.Pv,AIT504.Pv,PIT501.Pv,PIT502.Pv,PIT503.Pv,FIT601.Pv
0,2020-06-22 10:00:00,695.2841,0.0,0.000256,18.072288,7.519418,135.708786,7.482105,110.4296,40.335487,...,0.116122,0,7.615548,149.6283,46.91105,1.845681,232.376312,2.01839,206.9982,0.000256
1,2020-06-22 10:00:01,695.1271,0.0,0.000256,18.104332,7.514612,135.708786,7.482105,110.4296,40.335487,...,0.116122,0,7.612343,149.6283,46.91105,1.845681,232.376312,2.01839,206.9982,0.000256
2,2020-06-22 10:00:02.005000,694.930847,0.0,0.000256,18.104332,7.508844,135.708786,7.482105,110.4296,40.335487,...,0.116122,0,7.612343,149.6283,46.91105,1.845681,232.376312,2.01839,206.9982,0.000256
3,2020-06-22 10:00:03.010000,694.930847,0.0,0.000256,18.104332,7.504678,135.708786,7.489596,110.4296,40.335487,...,0.116122,0,7.612343,149.525757,46.91105,1.845681,232.376312,2.01839,206.9982,0.000256
4,2020-06-22 10:00:04.015000,694.8523,0.0,0.000256,18.104332,7.498911,135.708786,7.489596,110.4296,40.143414,...,0.116122,0,7.611382,149.525757,46.91105,1.845681,232.376312,2.01839,206.9982,0.000256


In [7]:
all_df.describe()

,AIT301.Pv,AIT302.Pv,AIT303.Pv
count,18000.000000,18000.000000,18000.000000
mean,7.683444,141.575679,47.199129
std,0.345545,31.024576,17.202684
min,6.571548,102.285675,29.003136
25%,7.396376,113.093025,40.047375
50%,7.629842,141.699219,44.128944
75%,7.992317,169.793200,50.579422
max,8.345636,209.949417,327.133600


In [8]:
all_df.dropna(axis=1, inplace=True)
all_df.describe(include='all')

,t_stamp,LIT101.Pv,FIT101.Pv,FIT201.Pv,AIT201.Pv,AIT202.Pv,AIT203.Pv,LIT301.Pv,FIT301.Pv,LIT401.Pv,...,FIT503.Pv,FIT504.Pv,AIT501.Pv,AIT502.Pv,AIT503.Pv,AIT504.Pv,PIT501.Pv,PIT502.Pv,PIT503.Pv,FIT601.Pv
count,32402,32402.0000,32402.0,32402.000000,32402.000000,32402.000000,32402.000000,32402.00000,32402.000000,32402.00000,...,32402.000000,32402,32402.000000,32402,32402.00000,32402.000000,32402.000000,32402.000000,32402.000000,32402.000000
unique,32402,5869.0000,307.0,1003.000000,1818.000000,5839.000000,2955.000000,5407.00000,1690.000000,7874.00000,...,128.000000,3,1370.000000,1283,614.00000,359.000000,800.000000,214.000000,782.000000,169.000000
top,2020-06-22 10:00:00,613.9524,0.0,0.000256,18.072288,9.640348,130.376831,1006.75311,0.000641,1006.20355,...,0.002176,0,7.264676,260,46.91105,1.691874,12.352602,2.050428,7.834529,0.000256
freq,1,271.0000,29387.0,11524.000000,3802.000000,49.000000,212.000000,1363.00000,10308.000000,296.00000,...,6936.000000,32400,110.000000,7200,3673.00000,2914.000000,1474.000000,4056.000000,938.000000,23697.000000


In [9]:
all_df.head()

,t_stamp,LIT101.Pv,FIT101.Pv,FIT201.Pv,AIT201.Pv,AIT202.Pv,AIT203.Pv,LIT301.Pv,FIT301.Pv,LIT401.Pv,...,FIT503.Pv,FIT504.Pv,AIT501.Pv,AIT502.Pv,AIT503.Pv,AIT504.Pv,PIT501.Pv,PIT502.Pv,PIT503.Pv,FIT601.Pv
0,2020-06-22 10:00:00,695.2841,0.0,0.000256,18.072288,7.519418,135.708786,1006.75311,0.000641,632.1841,...,0.116122,0,7.615548,149.6283,46.91105,1.845681,232.376312,2.01839,206.9982,0.000256
1,2020-06-22 10:00:01,695.1271,0.0,0.000256,18.104332,7.514612,135.708786,1006.75311,0.000641,631.838,...,0.116122,0,7.612343,149.6283,46.91105,1.845681,232.376312,2.01839,206.9982,0.000256
2,2020-06-22 10:00:02.005000,694.930847,0.0,0.000256,18.104332,7.508844,135.708786,1006.75311,0.000641,631.6842,...,0.116122,0,7.612343,149.6283,46.91105,1.845681,232.376312,2.01839,206.9982,0.000256
3,2020-06-22 10:00:03.010000,694.930847,0.0,0.000256,18.104332,7.504678,135.708786,1006.75311,0.000641,631.6073,...,0.116122,0,7.612343,149.525757,46.91105,1.845681,232.376312,2.01839,206.9982,0.000256
4,2020-06-22 10:00:04.015000,694.8523,0.0,0.000256,18.104332,7.498911,135.708786,1006.75311,0.000641,631.2997,...,0.116122,0,7.611382,149.525757,46.91105,1.845681,232.376312,2.01839,206.9982,0.000256


In [ ]:
if_outliers = get_list_of_if_outliers(all_df, 0.03)
print(f'Number of outliers: {len(if_outliers)}')